<a href="https://colab.research.google.com/github/Mahaveersalvih/Intelligent-Movie-Recommendation-System-Content-Based-Filtering-Mood-Analysis.ipynb/blob/main/Intelligent_Movie_Recommendation_System_Content_Based_Filtering_%26_Mood_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [1]:
!pip install streamlit
import numpy as np
import pandas as pd
import streamlit as st
import requests
import pickle
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.3 MB/s eta 0:00:00


In [2]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

In [3]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# number of rows and columns in the data frame

movies_data.shape

(4803, 24)

In [5]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director','id']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'id']


In [6]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [7]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']+' '+movies_data['id'].astype(str)

In [8]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [9]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [10]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [11]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 129068 stored elements and shape (4803, 22079)>
  Coords	Values
  (0, 4962)	0.07558463332496819
  (0, 5035)	0.08675091800727598
  (0, 10035)	0.10682369557117992
  (0, 18360)	0.0996650735240196
  (0, 10198)	0.0996650735240196
  (0, 8439)	0.20571442887524974
  (0, 7826)	0.21356327043425335
  (0, 10597)	0.15835713517009034
  (0, 19139)	0.32659730933843956
  (0, 21348)	0.12067958495774306
  (0, 7986)	0.24002536695991955
  (0, 19032)	0.20571442887524974
  (0, 9706)	0.23104072162345685
  (0, 20022)	0.0682359348921808
  (0, 21759)	0.12329360255434745
  (0, 15953)	0.08702131936522213
  (0, 16264)	0.2616731586704858
  (0, 18110)	0.14444955478119179
  (0, 21768)	0.227362220585816
  (0, 22051)	0.19422995678876445
  (0, 18080)	0.20939292991289057
  (0, 18825)	0.1980589697405917
  (0, 21429)	0.1908195360387445
  (0, 19369)	0.1456939887959868
  (0, 13517)	0.21837757524935328
  :	:
  (4801, 22027)	0.2762414077101103
  (4801, 18596)	0.26675

Cosine Similarity

In [12]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [13]:
print(similarity)

[[1.         0.06736808 0.03483192 ... 0.         0.         0.        ]
 [0.06736808 1.         0.03056728 ... 0.03330229 0.         0.        ]
 [0.03483192 0.03056728 1.         ... 0.         0.04952055 0.        ]
 ...
 [0.         0.03330229 0.         ... 1.         0.         0.02447526]
 [0.         0.         0.04952055 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02447526 0.         1.        ]]


In [14]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from the user

In [15]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : hulk


In [16]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [17]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Hulk']


In [18]:
close_match = find_close_match[0]
print(close_match)

Hulk


In [19]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

165


In [20]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.05600362124542374)), (1, np.float64(0.01899710341319054)), (2, np.float64(0.029749955799791977)), (3, np.float64(0.016129920075399744)), (4, np.float64(0.027350742140413338)), (5, np.float64(0.09389106678846988)), (6, np.float64(0.0)), (7, np.float64(0.02827595793656194)), (8, np.float64(0.0)), (9, np.float64(0.006180846075465187)), (10, np.float64(0.036904196667190525)), (11, np.float64(0.00560672036594873)), (12, np.float64(0.04088739731959254)), (13, np.float64(0.010535777187324437)), (14, np.float64(0.032936329909928515)), (15, np.float64(0.0)), (16, np.float64(0.02691578609089277)), (17, np.float64(0.006126186835532092)), (18, np.float64(0.07417027642534589)), (19, np.float64(0.014415146007752297)), (20, np.float64(0.01132033695821953)), (21, np.float64(0.0053140818296194)), (22, np.float64(0.004640995886213034)), (23, np.float64(0.063327286123357)), (24, np.float64(0.018645684096885794)), (25, np.float64(0.003235787445033972)), (26, np.float64(0.0291106696898635

In [21]:
len(similarity_score)

4803

In [22]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(165, np.float64(1.0000000000000002)), (2452, np.float64(0.14564782442595695)), (43, np.float64(0.142719841637812)), (4160, np.float64(0.13112898974391998)), (421, np.float64(0.12373947677532443)), (158, np.float64(0.11603775777806648)), (2394, np.float64(0.11551501435059058)), (2141, np.float64(0.11345952268610109)), (162, np.float64(0.11162483244346783)), (1920, np.float64(0.1107520344993242)), (747, np.float64(0.10835619002495146)), (1162, np.float64(0.10764408890311078)), (1913, np.float64(0.10524027371822066)), (816, np.float64(0.10522545038534523)), (1599, np.float64(0.10348360488856959)), (704, np.float64(0.10184510480344681)), (1434, np.float64(0.10181062075928984)), (1367, np.float64(0.1017791862978282)), (3668, np.float64(0.10127610503689816)), (1762, np.float64(0.09989889560141083)), (31, np.float64(0.09975781991004977)), (1959, np.float64(0.09957605859731081)), (1826, np.float64(0.09882918982423494)), (469, np.float64(0.09472637712944232)), (28, np.float64(0.09434516606607

In [23]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Hulk
2 . House of Sand and Fog
3 . Terminator Salvation
4 . Northfork
5 . Zodiac
6 . Star Trek
7 . Wolves
8 . Milk
9 . Stealth
10 . Warrior


Movie Recommendation Sytem

In [24]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<20):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : batman
Movies suggested for you : 

1 . Batman
2 . Batman Returns
3 . Batman & Robin
4 . The Dark Knight Rises
5 . Batman Begins
6 . The Dark Knight
7 . A History of Violence
8 . Superman
9 . Bedazzled
10 . Beetlejuice
11 . The Sentinel
12 . Mars Attacks!
13 . Planet of the Apes
14 . Man of Steel
15 . Suicide Squad
16 . Salton Sea
17 . The Mask
18 . Hang 'em High
19 . Spider-Man 3


In [25]:
import pickle
with open('movies.pkl','wb')as file:
    pickle.dump(movies_data,file)

with open('similarity.pkl','wb')as file:
    pickle.dump(similarity,file)

In [26]:
mood_to_genres = {
    'happy': ['Comedy', 'Family', 'Animation', 'Adventure'],
    'sad': ['Drama', 'Tragedy', 'Romance'],
    'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'],
    'romantic': ['Romance', 'Drama'],
    'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'],
    'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'],
    'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'],
    'funny': ['Comedy', 'Animation', 'Family']
}

print("Mood to genre mapping dictionary created:")
print(mood_to_genres)

Mood to genre mapping dictionary created:
{'happy': ['Comedy', 'Family', 'Animation', 'Adventure'], 'sad': ['Drama', 'Tragedy', 'Romance'], 'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'], 'romantic': ['Romance', 'Drama'], 'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'], 'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'], 'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'], 'funny': ['Comedy', 'Animation', 'Family']}


In [27]:
user_mood = input("Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): ")
print(f"You entered: {user_mood}")

Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): sad
You entered: sad


In [28]:
genres_for_mood = mood_to_genres.get(user_mood.lower(), [])

if not genres_for_mood:
    print(f"Mood '{user_mood}' not recognized. Please choose from: {', '.join(mood_to_genres.keys())}.")
    print("No genres retrieved for this mood.")
else:
    print(f"Genres associated with '{user_mood}': {genres_for_mood}")

Genres associated with 'sad': ['Drama', 'Tragedy', 'Romance']


In [29]:
import pandas as pd
import random

movies_data = pd.read_csv('/content/movies.csv')
movies_data['genres'] = movies_data['genres'].fillna('') # Fill NaN values with empty strings
mood_based_recommendations = []

if genres_for_mood:
    for index, row in movies_data.iterrows():
        movie_genres = row['genres'].split(' ')
        if any(genre in movie_genres for genre in genres_for_mood):
            mood_based_recommendations.append(index)

# Shuffle the recommendations to get different movies each time
random.shuffle(mood_based_recommendations)

print(f"Total movies found for '{user_mood}': {len(mood_based_recommendations)}")
print("First 10 recommended movie indices for this mood (shuffled):")
for i, movie_index in enumerate(mood_based_recommendations[:10]):
    title = movies_data.loc[movie_index, 'title']
    print(f"  {i+1}. {title} (Index: {movie_index})")

Total movies found for 'sad': 2586
First 10 recommended movie indices for this mood (shuffled):
  1. Being Julia (Index: 2364)
  2. Instructions Not Included (Index: 3562)
  3. Conspiracy Theory (Index: 455)
  4. Rent (Index: 1235)
  5. xXx: State of the Union (Index: 767)
  6. The Deep End of the Ocean (Index: 1264)
  7. The Four Seasons (Index: 3422)
  8. Promised Land (Index: 2620)
  9. Living Out Loud (Index: 2855)
  10. The To Do List (Index: 4229)


In [30]:
print(f"\nTop 15 Movies for your '{user_mood}' mood:")

if mood_based_recommendations:
    for i, movie_index in enumerate(mood_based_recommendations[:15]):
        title = movies_data.loc[movie_index, 'title']
        print(f"  {i+1}. {title}")
else:
    print("No movies found for this mood.")


Top 15 Movies for your 'sad' mood:
  1. Being Julia
  2. Instructions Not Included
  3. Conspiracy Theory
  4. Rent
  5. xXx: State of the Union
  6. The Deep End of the Ocean
  7. The Four Seasons
  8. Promised Land
  9. Living Out Loud
  10. The To Do List
  11. When Harry Met Sally...
  12. Changeling
  13. Bless the Child
  14. Trippin'
  15. Drinking Buddies


In [31]:
# 1. Install Streamlit
!pip install -q streamlit

# 2. Get your public IP (You will need this password shortly)
!wget -q -O - ipv4.icanhazip.com

34.74.178.234


In [1]:
# 3. Run Streamlit in the background and expose it via localtunnel
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼

⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.178.234:8501

y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴your url is: https://light-donuts-brush.loca.lt
  Stopping...
^C


In [33]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful